# Processamento de Linguagem Natural aplicada à Gestão Pública

Aula 5 (08/06): Modelagem de Tópicos, Classificação de Textos e Reconhecimento de entidades nomeadas

Reconhecimento de menção a pessoas, lugares e organizações e outras entidades. Classificação de Textos. Identificação de tópicos. Alocação Latente de Dirichlet.



## Preparação do ambiente

In [1]:
# Download dos dados
%%capture
!pip install spacy==3.3.0
!git clone https://github.com/samuelbarbosaa/oficina_nlp.git

import gdown
import tarfile
import spacy

url = 'https://drive.google.com/uc?id=1-zEjxcpyd3OwBI1Uve90MMcIkjRPH7FR'
output = 'pt_core_news_lg-3.3.0.tar.gz'
gdown.download(url, output, quiet=False)

file = tarfile.open('pt_core_news_lg-3.3.0.tar.gz')
file.extractall('./')
file.close()

nlp = spacy.load("/content/pt_core_news_lg-3.3.0/pt_core_news_lg/pt_core_news_lg-3.3.0")

In [2]:
import pickle
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

pd.options.display.max_rows = 6
plt.rcParams['figure.figsize'] = [10, 5]

In [3]:
import zipfile

with zipfile.ZipFile("/content/oficina_nlp/data/acordaos_cpf.zip", "r") as zip_ref:
    zip_ref.extractall("/content/oficina_nlp/data/")

acordaos = pd.read_csv("/content/oficina_nlp/data/acordaos_cpf.csv")
acordaos

,ano_acordao,numero_acordao,relator,processo,tipo_processo,assunto,sumario,acordao
0,2000,322,ADYLSON MOTTA,575.497/1995-5,II - Tomada de Contas Especial,II - Tomada de Contas Especial,Tomada de Contas Especial. Subvenção social co...,"VISTOS, relatados e discutidos estes autos de ..."
1,2000,323,ADYLSON MOTTA,575.191/1996-1,II - Tomada de Contas Especial,II - Tomada de Contas Especial,Tomada de Contas Especial. Subvenção social co...,"VISTOS, relatados e discutidos estes autos de ..."
2,2000,292,JOSÉ ANTÔNIO BARRETO DE MACEDO,275.267/1995-4,IV - Relatório de Auditoria transformado em to...,IV - Relatório de Auditoria transformado em to...,Relatório de Auditoria transformado em tomada ...,"VISTOS, relatados e discutidos estes autos de ..."
...,...,...,...,...,...,...,...,...
29910,2018,11377,AUGUSTO NARDES,027.687/2011-2,NaN,NaN,NaN,ACÓRDÃO Nº 11377/2018 - TCU - 2ª Câmara ...
29911,2018,11633,AUGUSTO NARDES,032.564/2011-2,NaN,NaN,NaN,ACÓRDÃO Nº 11633/2018 - TCU - 2ª Câmara ...
29912,2018,2977,AUGUSTO NARDES,011.645/2018-0,Auditoria com o objetivo de verificar a aplica...,Auditoria com o objetivo de verificar a aplica...,RELATÓRIO DE AUDITORIA. ATENDIMENTO DE SOLICIT...,"VISTO, relatado e discutido este processo de f..."


In [4]:
import nltk
import string
import re
from spacy.lang.pt import STOP_WORDS as spacy_stopwords
from functools import reduce

nltk.download('stopwords')
nltk_stopwords = nltk.corpus.stopwords.words("portuguese")
all_sw = list(spacy_stopwords) + nltk_stopwords

def remove_pontuacao(x):
    punct = [s for s in string.punctuation] + ["º"]
    return reduce(lambda w, s: w.replace(s, ""), punct, x)

def remove_stopwords(x):
    palavras = x.split(" ")
    return " ".join([w for w in palavras if w.lower() not in all_sw])

def remove_espacos_extras(x):
    return re.sub(r"\s+", " ", x)

def remove_ngram(x, n):
    palavras = x.split(" ")
    return " ".join([w for w in palavras if not len(w) <= n])

def padronizar_texto(x):
    y = re.sub(pattern=r"\W", repl=" ", string=x)
    y = re.sub(pattern=r"\d", repl="", string=y)
    y = re.sub(pattern=r"\n\r+", repl="", string=y)
    y = remove_pontuacao(y)
    y = remove_stopwords(y)
    y = remove_espacos_extras(y)
    y = remove_ngram(y, 2)
    return y.lower()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


# REMOVER NOMES PROPRIOS

In [5]:
#acordaos["acordao_padr"] = acordaos["acordao"].apply(padronizar_texto)

In [6]:
#docs = list(nlp.pipe(acordaos["acordao_padr"], disable=['parser', 'ner']))

#def lemmatizar(doc):
#  lemmas = [w.lemma_ for w in doc if w.pos_ != "PROPN"]
#  return " ".join(lemmas)

#acordaos["acordao_padr"] = [lemmatizar(doc) for doc in docs]
#acordaos

In [7]:
import pickle

#with open("/content/oficina_nlp/acordaos.pkl", "wb") as fp:
#  pickle.dump(acordaos, fp)

with open("/content/oficina_nlp/acordaos.pkl", "rb") as fp:
  acordaos = pickle.load(fp)

acordaos

FileNotFoundError: ignored

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

metodo = TfidfVectorizer()
modelo = metodo.fit(acordaos["acordao_padr"])
vocab = modelo.get_feature_names_out()
TF_IDF = modelo.transform(acordaos["acordao_padr"])
TF_IDF

In [ ]:
from sklearn.decomposition import LatentDirichletAllocation

lda = LatentDirichletAllocation(n_components=30, max_iter=200)
topicos = lda.fit_transform(TF_IDF)
palavras_topicos = lda.components_

with open("/content/oficina_nlp/lda.pkl", "wb") as fp:
  pickle.dump(lda, fp)

with open("/content/oficina_nlp/lda.pkl", "rb") as fp:
  lda = pickle.load(fp)

In [ ]:
%%capture
!pip install pyldavis

In [ ]:
import pyLDAvis
import pyLDAvis.sklearn

pyLDAvis.enable_notebook()
pyLDAvis.sklearn.prepare(lda, TF_IDF, metodo, mds="tsne", R=15)